# Setup and run simple model


In this notebook, we will outline how to define and run a very simple hydrologic model


In [2]:
import pandas as pd
import potions as pt
import matplotlib.pyplot as plt

## Load the input data and create forcing data


The first step in any modeling workflow is to prepare the input data. For a hydrologic model, this data is often called **forcing data**, as it 'forces' the model to simulate the catchment's response. This data consists of meteorological variables that drive the water balance that we don't simulate within the model.

For this example, we are using data from the Sleeper's River research watershed, compiled by Stewart et al. (2024). The provided file also contains simulation results from another model (HBV-light), but for our purposes, we will only use the raw weather data and the observed streamflow (`Qobs`).

A `potions` hydrologic model requires the following forcing variables:

- **Precipitation**: The amount of water input to the catchment (e.g., rainfall, snow) in units of (mm/$\Delta t$).
- **Temperature**: Air temperature in Celsius, which is crucial for the snow routines in the model.
- **Potential Evapotranspiration (PET)**: A measure of the 'drying power' of the atmosphere, representing the amount of water that could be evaporated and transpired if sufficient water were available. Like precipitation, this also has units of $\frac{mm \text{ water}}{\Delta t}$
  - This can be estimated using an empirical equation, from simple equations like the Hargreaves or Hamon equations up to the Penman-Monteith equation.

We also need measured stream discharge data for the site that we are modeling if we want to be able to calibrate the model. Like precipitation and PET, the stream discharge must be in units of millimeters per day. While these units might be strange, are really just the amount of water per units area in the catchment. To get from commonly measured units like cubic feet per second, just use the follwoing conversion:

$$
1 \frac{mm}{d} = \frac{1 ft^3}{sec}
\cdot \frac{0.0283 \cancel{m^2} \cdot \cancel{m}}{ft^3}
\cdot \frac{86400 sec}{day}
\cdot \frac{1}{\text{Area} \; \cancel{km^2}}
\cdot \frac{1 \cancel{km^2}}{10^6 \cancel{m^2}}
\cdot \frac{1000 mm}{\cancel{m}} = \frac{2.445 ft^3}{A \cdot sec}
$$

In other words, multiply the streamflow time series by a factor $\frac{2.445}{A}$, where $A$ is the catchment drainage area in square kilometers, and you end up with the correct streamflow units.

To prepare this data for the model, we first load it into a `pandas` DataFrame. It is critical that the DataFrame's index is a datetime object and that all data series are at the same time interval (e.g., daily, hourly, ...). We will then use this DataFrame to create a `potions.ForcingData` object, which is the required input format for running a `potions` hydrologic model.


In [ ]:
data_path: str = "../input/Sleepers_Results.txt"
input_df: pd.DataFrame = pd.read_csv(
    data_path, sep="\\s+", index_col="Date", parse_dates=True
)

In [4]:
# Now, create the ForcingData object
forcing: pt.ForcingData = pt.ForcingData(
    precip=input_df["Precipitation"],
    temp=input_df["Temperature"],
    pet=input_df["PET"],
)
meas_streamflow: pd.Series = input_df["Qobs"]

## Create a hydrologic model

Now that we have our forcing data, we can create a hydrologic model. A hydrologic model is a simplified, conceptual representation of the water cycle processes in a catchment. It takes forcing data (like precipitation and temperature) as input and simulates how water moves through different stores (like snow, soil, and groundwater) to produce streamflow.

For this example, we will use the `HbvModel`, which is an implementation of the popular HBV (Hydrologiska Byråns Vattenbalansavdelning) model. This model is widely used in hydrology for its relative simplicity and good performance in many environments.

The Potions modeling framework conceptualizes a catchment as a series of vertically stacked layers, where each layer represents a distinct part of the hydrologic system. These layers are composed of one or more "zones" (HydrologicZone), which act as individual computational units or conceptual buckets. Water moves between these zones both vertically, from an upper layer to the one below it, and laterally between zones within the same layer. The model calculates the complete water balance for each zone at every time step, simulating processes like infiltration, evapotranspiration, and runoff generation. For example, the HbvModel is structured with four distinct layers: a SnowZone at the top to accumulate and melt snow, a SoilZone beneath it to handle soil moisture, followed by a GroundZoneLinear for shallow groundwater, and finally a GroundZoneLinearB at the bottom representing a deep groundwater reservoir. This layered structure allows the model to simulate the sequential flow of water through the critical components of a catchment.


In [7]:
model: pt.HbvModel = pt.HbvModel()

# View the zone names in the model
print("Model Zones:", model.get_zone_names())

Model Zones: ['snow', 'soil', 'shallow', 'deep']


## Modify model parameters

Hydrologic models have parameters that control the behavior of the simulated processes. These parameters are tunable constants that define the size and behavior of the conceptual stores in our model. For example, a parameter might define the maximum water storage in the soil, while another controls how quickly snow melts. Since these are conceptual representations, the "true" values are often unknown and must be estimated by fitting the model to observed data, a process called calibration.

First, let's inspect the default parameters of the `HbvModel` we created. We can get all the model's parameters as a dictionary. The keys are formatted as `zone_name.parameter_name`.


In [11]:
print("Default parameters:")
for param_name, param_val in model.to_dict().items():
    print(f"  {param_name}: {param_val}")

Default parameters:
  snow.tt: 0.0
  snow.fmax: 1.0
  soil.fc: 100.0
  soil.lp: 0.5
  soil.beta: 1.0
  soil.k0: 0.1
  soil.thr: 10.0
  shallow.k: 0.01
  shallow.perc: 1.0
  deep.k: 0.01


To alter the model parameters, get the zone object using `model[<zone_name>]`, and set the parameters on that object using the zone names that we saw earlier. Just for reference again, the zones on the `HbvModel` are `snow`, `soil`, `shallow`, and `deep`:


In [13]:
model[
    "snow"
].tt = 0.1  # Set the snow melting point to 0.1 degrees C, which might be the effective melting point
model["soil"].fc = 250.0  # Set the soil field capacity to 250 mm
model["shallow"].perc = 0.5  # Set the maximum percolation rate to 0.5 mm/day
model["deep"].k = 1e-3  # Set the deep groundwater recession constant to be much smaller

## Run the model and visualize the outputs

With our forcing data and model (with its parameters), we can now run the simulation. The model will process the forcing data day by day and compute the resulting streamflow. Since we have measured streamflow, the model will also calculate some goodness of fit functions so that we can observe how well our model fits the real data.

After running the model, a good practice is to visualize the output. We will plot the simulated streamflow against the observed streamflow (which is in our input data) to see how well our model performs with the chosen parameters.


In [ ]:
# Run the model
results: pd.DataFrame = model.run(forcing)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(results.index, results["q_sim"], label="Simulated Streamflow")
plt.plot(input_df.index, input_df["Qobs"], label="Observed Streamflow", linestyle="--")
plt.title("Simulated vs. Observed Streamflow")
plt.xlabel("Date")
plt.ylabel("Streamflow (mm/day)")
plt.legend()
plt.show()

## Save the model results

Finally, you might want to save the results of your model run for further analysis or for use in other applications. The results are returned as a `pandas` DataFrame, which can be easily saved to various formats, like a CSV file.


In [ ]:
output_path: str = "../output/model_results.csv"
results.to_csv(output_path)

print(f"Model results saved to {output_path}")